# \#SAT by Boolean Tensor Contraction

This notebook contains a short, self-contained explanation of #SAT computation using Boolean tensor network contraction.

We implement Boolean tensors as NumPy UInt8 arrays with entries constrained to take values 0 or 1.

In [49]:
from collections.abc import Sequence
from typing import Any
import numpy as np

type IntArrayLike = Sequence[int] | Sequence[IntArrayLike]
type BoolTensor = np.ndarray[Any, np.dtype[np.uint8]]
def bool_tensor(data: IntArrayLike) -> BoolTensor:
    return np.sign(np.array(data, np.uint8))

We instantiate Boolean tensors for the basic building blocks of SAT problems.

In [62]:
# Constants bit values:
false = bool_tensor([
    1 if in_ == 0 else 0
    for in_ in [0, 1]
])
true = bool_tensor([
    1 if in_ == 1 else 0
    for in_ in [0, 1]
])


# Boolean operators:
not_ = bool_tensor([
    [
        1 if out_ == 1-in_ else 0
        for out_ in [0, 1]
    ]
    for in_ in [0, 1]
])
and_ = bool_tensor([
    [
        [
            1 if out_ == inl&inr else 0
            for out_ in [0, 1]
        ]
        for inr in [0, 1]
    ]
    for inl in [0, 1]
])
or_ = bool_tensor([
    [
        [
            1 if out_ == inl|inr else 0
            for out_ in [0, 1]
        ]
        for inr in [0, 1]
    ]
    for inl in [0, 1]
])
xor_ = bool_tensor([
    [
        [
            1 if out_ == inl^inr else 0
            for out_ in [0, 1]
        ]
        for inr in [0, 1]
    ]
    for inl in [0, 1]
])

Boolean tensors are indicator functions for subsets of the Cartesian products of finite, explicitly enumerated sets.
Specifically, the Boolean tensor of shape $(n_0,...,n_{k-1})$ are the indicator functions for the subsets of:

$$
\prod_{j=0}^{k-1} \lbrace 0, ..., n_j-1 \rbrace
$$

Below is a utility function which converts a given Boolean tensor to the corresponding subset.

In [52]:
def to_subset(tensor: BoolTensor) -> frozenset[tuple[int, ...]]:
    return frozenset(
        idxs
        for idxs in np.ndindex(tensor.shape)
        if tensor[*idxs]
    )

In [59]:
print(f"{and_.shape = }")
# (2, 2, 2)
#        ^ 1 output bit
#  ^^^^ 2 input bits

and_.shape = (2, 2, 2)


In [60]:
print(f"{to_subset(and_) = }")
# { (0 , 0 , 0), (0 , 1 , 0), (1 , 0 , 0), (1 , 1 , 1) }
#    0 & 0 = 0    0 & 1 = 0    1 & 0 = 0    1 & 1 = 1

to_subset(and_) = frozenset({(1, 0, 0), (0, 0, 0), (1, 1, 1), (0, 1, 0)})


We can combine the Boolean constants and operator into a tensor network, representing an arbitrary formula.
We can then perform tensor network contraction to compute different quantities:

- Leaving the inputs open, we compute Boolean tensor corresponding to the indicator function of all the satisfying assignments.
- Discarding the inputs, we compute the number of satisfying assignments.

The counting argument works because the formula takes the form of an equation between functions: for a given assignment of values to the inputs, there is exactly one corresponding value that each internal variable can take.

Below are the [NumPy einsum](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html) strings for both contractions, in the case of the following program:

```
inputs: a, b, c
d = a or b
e = d or c
assert e is true
f = not c
g = not b
h = not a
i = f or g
j = i or h
assert j is true
```

The above corresponds to requiring satisfaction of the following formula:

```
((a|b)|c) & ((~c|~b)|~a)
```

Note that the conjunction is implicit, because it corresponds to scalar multiplication in the dagger compact category of sets and relations (i.e. to scalar multiplication for Boolean tensors).

In [65]:
arrays = [or_, or_, true, not_, not_, not_, or_, or_, true]
#         abd  dce  e     cf    bg    ah    fgi  ihj  j
count_sat_einsum_expr = "abd,dce,e,cf,bg,ah,fgi,ihj,j->"
all_sat_ensum_expr = "abd,dce,e,cf,bg,ah,fgi,ihj,j->abc"

In [66]:
print("Number of satisfying assignments:")
np.einsum(count_sat_einsum_expr, *arrays)

Number of satisfying assignments:


np.uint8(6)

In [67]:
print("Satisfying assignments:")
for i, b in enumerate(np.einsum(all_sat_ensum_expr, *arrays).reshape(8)):
    print(f"{i:03b} -> {b}")

Satisfying assignments:
000 -> 0
001 -> 1
010 -> 1
011 -> 1
100 -> 1
101 -> 1
110 -> 1
111 -> 0
